In [1]:
import numpy as np
from ReplayBuffer import ReplayBuffer
from TD3 import TD3
from Critic import Critic
from Actor import Actor
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler


In [2]:
state_dim = 6
action_dim = 6
max_time = 10

In [3]:
rb = ReplayBuffer(state_dim,action_dim)

In [4]:
# policy = TD3(state_dim,action_dim,max_time,100,100)

In [5]:
act = Actor(state_dim,action_dim,max_time,0.7,5).to('cpu')

In [6]:
for i in range(100):
    state = np.arange(6)
    action = np.arange(6)
    nxt_state = np.arange(6)
    reward = np.random.uniform(1000)
    rb.store(state,action,nxt_state,reward)

In [7]:
state, action, nxt_state, reward = rb.sample(32)

In [8]:
state[0]

array([0., 1., 2., 3., 4., 5.])

In [9]:
act(torch.Tensor(state[0]))

tensor([0.2800, 0.0284, 0.0127, 0.4628, 0.0867, 0.0100],
       grad_fn=<AbsBackward0>)


tensor([[2.8001, 0.0853, 0.1138, 0.0089, 2.3142, 0.4333, 0.0501]],
       grad_fn=<CatBackward0>)

In [7]:
policy.train(rb,10)

In [9]:
action = np.random.rand(19)

In [13]:
T_th = [np.abs(10 * action[0])]
coef = 500 * action[1:]
action = np.concatenate((T_th,coef))

In [14]:
action

array([  3.3242602 , 362.18142335, 238.05038436, 137.94959479,
       305.62622241,  11.29523593, 189.72405898, 164.5146315 ,
       319.40553002, 409.12145127, 226.30330379, 147.13353829,
        41.28369998, 465.91540901, 111.55983097, 199.7546455 ,
        86.58571038, 116.12232831, 317.70933147])

In [ ]:
from ReplayBuffer import ReplayBuffer
import os
import matplotlib.pyplot as plt


In [ ]:
rb = ReplayBuffer(state_dim,action_dim,'../../../../../../ReplayBuffer.csv')

In [ ]:
data = rb.dump()

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Reward over episode")
plt.xlabel("Episode#")
plt.ylabel("Reward")
plt.plot(data['reward'], color="orange", linewidth=2.5)
plt.savefig('test.jpg')
plt.show()